In [4]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

# Load videos from the folder
def load_videos_from_folder(folder_path):
    videos = []
    labels = []
    
    # List all action categories
    action_labels = os.listdir(folder_path)
    
    for label in action_labels:
        action_folder = os.path.join(folder_path, label)
        for video_file in os.listdir(action_folder):
            if video_file.endswith('.avi'):  # Assuming videos are in .avi format
                video_path = os.path.join(action_folder, video_file)
                videos.append(video_path)
                labels.append(label)  # Store the corresponding label
                
    return videos, labels

# Preprocess the video
def preprocess_video(video_path, num_frames=30, img_size=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames == 0:
        print(f"Warning: No frames found in video {video_path}")
        return np.array([])  # Return an empty array if no frames are found

    step = total_frames // num_frames
    
    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * step)  # Set the frame position
        ret, frame = cap.read()
        if not ret:
            print(f"Warning: Could not read frame {i} from video {video_path}")
            break
        frame = cv2.resize(frame, img_size)  # Resize frame
        frames.append(frame)
    
    cap.release()
    return np.array(frames)

# Create dataset
def create_dataset(video_paths, labels):
    X = []
    y = []
    label_mapping = {label: idx for idx, label in enumerate(set(labels))}
    
    for video_path, label in zip(video_paths, labels):
        frames = preprocess_video(video_path)
        if frames.shape[0] > 0:
            X.append(frames)
            y.append(label_mapping[label])  # Convert label to numerical format
            
    return np.array(X), np.array(y)

# Build the model
def build_model(num_classes):
    base_model = tf.keras.applications.MobileNetV2(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
    base_model.trainable = False  # Freeze the base model

    # Define a feature extractor
    def feature_extractor(x):
        return base_model(x)

    # Input layer for the sequence of frames
    inputs = layers.Input(shape=(30, 224, 224, 3))
    x = layers.TimeDistributed(layers.Lambda(feature_extractor))(inputs)
    x = layers.TimeDistributed(layers.GlobalAveragePooling2D())(x)
    x = layers.LSTM(64)(x)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

# Load and preprocess dataset
video_folder_path = 'D:\pk\h1'  # Replace with the path to your HMDB51 dataset
video_paths, labels = load_videos_from_folder(video_folder_path)
X, y = create_dataset(video_paths, labels)

# Normalize the data
X = X.astype('float32') / 255.0

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build and compile the model
num_classes = len(set(y))
model = build_model(num_classes)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

# Classify a new video
def classify_video(video_path, model):
    frames = preprocess_video(video_path)
    if frames.size == 0:
        print("Error: No frames to classify.")
        return None
    
    frames = frames.astype('float32') / 255.0
    frames = np.expand_dims(frames, axis=0)  # Add batch dimension
    predictions = model.predict(frames)
    predicted_class = np.argmax(predictions, axis=1)
    return predicted_class

# Example usage of classify_video
new_video_path = 'h1/brush_hair/testing_1.avi'  # Replace with your video path
if not os.path.isfile(new_video_path):
    print(f"Error: The video file {new_video_path} does not exist.")
else:
    print("Predicted Class:", classify_video(new_video_path, model))


<>:85: SyntaxWarning: invalid escape sequence '\p'
<>:85: SyntaxWarning: invalid escape sequence '\p'
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_4600\1298831468.py:85: SyntaxWarning: invalid escape sequence '\p'
  video_folder_path = 'D:\pk\h1'  # Replace with the path to your HMDB51 dataset


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 163s 163s/step - accuracy: 0.3750 - loss: 0.7221 - val_accuracy: 1.0000 - val_loss: 0.4692
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - accuracy: 1.0000 - loss: 0.2488 - val_accuracy: 1.0000 - val_loss: 0.2983
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - accuracy: 1.0000 - loss: 0.0933 - val_accuracy: 1.0000 - val_loss: 0.2144
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - accuracy: 1.0000 - loss: 0.0421 - val_accuracy: 1.0000 - val_loss: 0.2017
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - accuracy: 1.0000 - loss: 0.0262 - val_accuracy: 1.0000 - val_loss: 0.1633
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 1.0000 - loss: 0.0155 - val_accuracy: 1.0000 - val_loss: 0.1394
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 1.0000 - loss: 0.0106 - val_accuracy: 1.0000 - val_loss: 0.1363
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 1.0000 - loss: 0.0075 - val_accuracy: 1.0000 - val_loss: 0.141